### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [ ]:
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 69.1 MB/s 
     |████████████████████████████████| 87 kB 7.4 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 6.4 MB 51.0 MB/s 
     |████████████████████████████████| 234 kB 72.3 MB/s 
     |████████████████████████████████| 1.2 MB 59.2 MB/s 
     |████████████████████████████████| 840 kB 60.1 MB/s 
     |████████████████████████████████| 1.1 MB 63.6 MB/s 
     |████████████████████████████████| 3.4 MB 58.1 MB/s 
     |████████████████████████████████| 120 kB 74.8 MB/s 
     |████████████████████████████████| 1.1 MB 55.9 MB/s 
     |████████████████████████████████| 462 kB 62.6 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 11.9 MB/s 
     |████████████████████████████████| 352 kB 69.3 MB/s 
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |██████████████

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Prepare the dataset


In [ ]:
# Install Kaggle API
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 74 kB 2.2 MB/s 
     |████████████████████████████████| 4.2 MB 11.4 MB/s 
     |████████████████████████████████| 150 kB 43.9 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 113 kB 50.0 MB/s 
  ERROR: Failed building wheel for lxml
    Running setup.py install for lxml ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ffma1xl8/lxml_2bd3c30a0b384808ad1eba09f9432796/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ffma1xl8/lxml_2bd3c30a0b384808ad1eba09f9432796/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-qq66s0ac/install-record.txt --s

In [ ]:
# only for google colab
import os
os.environ['KAGGLE_USERNAME'] = "<username>" 
os.environ['KAGGLE_KEY'] = "<key>"

In [ ]:
!kaggle datasets download -d tannergi/microcontroller-detection --unzip

401 - Unauthorized


## Train your object detection model

There are six steps to training an object detection model:

**Step 1. Choose an object detection model archiecture.**

This tutorial uses the EfficientDet-Lite2 model. EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture. 

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |




In [ ]:
spec = model_spec.get('efficientdet_lite3')

**Step 2. Load the dataset.**

Model Maker can load data from multiple different formats, including [CSV format](https://cloud.google.com/vision/automl/object-detection/docs/csv-format) and [Pascal VOC](https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5). For more information, check out the [`tflite_model_maker.object_detector.DataLoader` documentation](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/DataLoader).

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc('/content/train', '/content/train', ['drone'])

In [ ]:
validation_data = object_detector.DataLoader.from_pascal_voc('/content/test', '/content/test', ['drone'])

**Step 3. Train the TensorFlow model with the training data.**

In [ ]:
model = object_detector.create(train_data, model_spec=spec, epochs=40, batch_size=8, train_whole_model=True, validation_data=validation_data)

Epoch 1/40
96/96 [==============================] - 125s 840ms/step - det_loss: 0.8875 - cls_loss: 0.5472 - box_loss: 0.0068 - reg_l2_loss: 0.0926 - loss: 0.9802 - learning_rate: 0.0090 - gradient_norm: 4.1888 - val_det_loss: 0.6226 - val_cls_loss: 0.3955 - val_box_loss: 0.0045 - val_reg_l2_loss: 0.0928 - val_loss: 0.7155
Epoch 2/40
96/96 [==============================] - 77s 807ms/step - det_loss: 0.4537 - cls_loss: 0.2640 - box_loss: 0.0038 - reg_l2_loss: 0.0930 - loss: 0.5467 - learning_rate: 0.0100 - gradient_norm: 4.2147 - val_det_loss: 0.5766 - val_cls_loss: 0.3009 - val_box_loss: 0.0055 - val_reg_l2_loss: 0.0931 - val_loss: 0.6698
Epoch 3/40
96/96 [==============================] - 77s 799ms/step - det_loss: 0.4022 - cls_loss: 0.2375 - box_loss: 0.0033 - reg_l2_loss: 0.0933 - loss: 0.4955 - learning_rate: 0.0099 - gradient_norm: 4.5116 - val_det_loss: 0.5178 - val_cls_loss: 0.3432 - val_box_loss: 0.0035 - val_reg_l2_loss: 0.0935 - val_loss: 0.6113
Epoch 4/40
96/96 [============

**Step 4. Evaluate the model with the test data.**

After training the object detection model using the images in the training dataset, use the remaining 25 images in the test dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 7 images in the test dataset.

In [ ]:
model.evaluate(validation_data)

8/8 [==============================] - 27s 2s/step



{'AP': 0.62488,
 'AP50': 0.94523716,
 'AP75': 0.73806816,
 'AP_/drone': 0.62488,
 'APl': 0.6724523,
 'APm': 0.44989026,
 'APs': 0.12042438,
 'ARl': 0.76971155,
 'ARm': 0.62222224,
 'ARmax1': 0.649904,
 'ARmax10': 0.72380036,
 'ARmax100': 0.7353167,
 'ARs': 0.46}

**Step 5.  Export as a TensorFlow Lite model.**

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is full integer quantization.

In [ ]:
model.export(export_dir='/content')

**Step 6.  Evaluate the TensorFlow Lite model.**

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop. 
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('/content/model', validation_data)

ValueError: ignored

## (Optional) Test the TFLite model on your image

In [ ]:
!pip install opencv-python-headless==4.1.2.30

     |████████████████████████████████| 21.8 MB 787 kB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.62
    Uninstalling opencv-python-headless-4.5.5.62:
      Successfully uninstalled opencv-python-headless-4.5.5.62


In [ ]:
import cv2

from PIL import Image

model_path = '/content/model_ef0_ep30_bs8.tflite'

# Load the labels into a list
classes = ['drone'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  return resized_img, original_image


def set_input_tensor(interpreter, image):
  """Set the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image


def get_output_tensor(interpreter, index):
  """Retur the output tensor at the given index."""
  output_details = interpreter.get_output_details()[index]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""
  # Feed the input image to the model
  set_input_tensor(interpreter, image)
  interpreter.invoke()

  # Get all outputs from the model
  scores = get_output_tensor(interpreter, 0)
  boxes = get_output_tensor(interpreter, 1)
  count = int(get_output_tensor(interpreter, 2))
  classes = get_output_tensor(interpreter, 3)

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path, 
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute 
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
INPUT_IMAGE_URL = "/content/WhatsApp Image 2022-02-10 at 6.01.36 PM.jpeg"
DETECTION_THRESHOLD = 0.5 

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    INPUT_IMAGE_URL, 
    interpreter, 
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)

                                                                                                                                      niyaskhan1011